In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("cleanup_media.csv")

In [3]:
import boto
import boto3
conn = boto.connect_s3()
import requests

In [4]:
# Uses the creds in ~/.aws/credentials
s3 = boto3.resource('s3')
bucket_name_to_upload_image_to = 'mediacleanup'

In [5]:
# Do this as a quick and easy check to make sure your S3 access is OK
for bucket in s3.buckets.all():
    if bucket.name == bucket_name_to_upload_image_to:
        print('Good to go. Found the bucket to upload the image into.')
        good_to_go = True

if not good_to_go:
    print('Not seeing your s3 bucket, might want to double check permissions in IAM')

Good to go. Found the bucket to upload the image into.


In [6]:
mapping_dict ={}
for i, img_url in enumerate(df["image_url"]):
    img_name = "mediacleanup_%05d" % (i,)
    mapping_dict[img_name] = img_url
    
    if (img_url == np.nan) | (str(img_url) == "nan"):
        continue
    else:
        # Uses the creds in ~/.aws/credentials
        s3_image_filename = img_name
        internet_image_url = img_url

        # Given an Internet-accessible URL, download the image and upload it to S3,
        # without needing to persist the image to disk locally
        req_for_image = requests.get(internet_image_url, stream=True)
        file_object_from_req = req_for_image.raw
        req_data = file_object_from_req.read()

        # Do the actual upload to s3
        s3.Bucket(bucket_name_to_upload_image_to).put_object(Key=s3_image_filename, Body=req_data)

In [7]:
md_01 = pd.DataFrame(mapping_dict, index = range(0,len(mapping_dict)))
md_02 = pd.DataFrame(md_01.T[0])
pd.DataFrame(md_02).to_csv('../assets/media_cleanup.csv', index = range(0,len(mapping_dict)))

In [23]:
-

In [24]:
img_df_long

,img,type,label,confidence
0,mediacleanup_00000,Label,Human,99.283340
1,mediacleanup_00000,Label,People,99.283340
2,mediacleanup_00000,Label,Person,99.283340
3,mediacleanup_00000,Label,Architecture,75.759644
4,mediacleanup_00000,Label,Building,75.759644
5,mediacleanup_00000,Label,Skylight,75.759644
6,mediacleanup_00000,Label,Window,75.759644
7,mediacleanup_00000,Text,SELF,98.684273
8,mediacleanup_00000,Text,Instagram: @lilmistydiaz,93.729767
9,mediacleanup_00000,Text,SELF,98.684273


In [25]:
len(img_df_long)

33

In [20]:
len(mapping_dict)

14

In [21]:
len(img_df_long)

33

In [13]:
# for-loop to drop all "Human", "People", "Person" Label rows where the image 
# contains a celebrity.
for img in img_df_long["img"]:
    if len(img_df_long.loc[(img_df_long["img"] == img) & (img_df_long["type"]=="Celebrity")])>0:
        img_df_long = img_df_long.loc[~((img_df_long['img'] == img) 
                          & (img_df_long['label'].isin(['Human', 'People', 'Person'])))]

In [14]:
# for-loop to drop all "Label" rows below 90% confidence if there is a celebrity
for img in img_df_long["img"]:
    if len(img_df_long.loc[(img_df_long["img"] == img) & (img_df_long["type"]=="Celebrity")])>0:
        img_df_long = img_df_long.loc[~((img_df_long['img'] == img) 
                          & (img_df_long['type'].isin(['Label'])) & (img_df_long['confidence']<90))]

In [15]:
# for loop to drop all "Label", "Sticker", "Text" label rows where image contains text.
for img in img_df_long["img"]:
    if len(img_df_long.loc[(img_df_long["img"] == img) & (img_df_long["type"]=="Text")])>0:
        img_df_long = img_df_long.loc[~((img_df_long['img'] == img) 
                          & (img_df_long['label'].isin(['Label', 'Sticker', 'Text'])))]

In [16]:
media_new = []
for img in img_df_long['img'].unique():
    img_dict = {'img': img}
    if len(img_df_long[(img_df_long['img']==img) & (img_df_long['type']=='Label')])>0:
        img_dict['label'] = ' '.join(img_df_long.loc[(img_df_long['img']==img) & (img_df_long['type']=='Label'), 'label'].tolist())
    else:
        img_dict['label'] = None
    if len(img_df_long[(img_df_long['img']==img) & (img_df_long['type']=='Text')])>0:
        text = [str(detected_text) 
                for detected_text in img_df_long.loc[(img_df_long['img']==img) & (img_df_long['type']=='Text'), 'label'].tolist()]
        img_dict['text'] = ' '.join(text)
    else:
        img_dict['text'] = None
    img_dict['celebrity'] = len(img_df_long[(img_df_long['img']==img) & (img_df_long['type']=='Celebrity')])>0
    media_new.append(img_dict)
media_new_df = pd.DataFrame(media_new)

In [18]:
media_new_df

,celebrity,img,label,text
0,False,mediacleanup_00000,Human People Person Architecture Building Skyl...,SELF Instagram: @lilmistydiaz SELF Instagram: ...
1,False,mediacleanup_00001,Human People Person Premiere Red Carpet Red Ca...,VM VMA penoom VM 2017 VI0EO MUSIC K VMA V VMA ...


In [27]:
media_new_df.to_csv("01_media_cleanup_text_data.csv")